<a href="https://colab.research.google.com/github/joaohnt/estudo_ml/blob/main/OneR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Orange3

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 534.6/534.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.0/125.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95

In [3]:
import Orange

# base risco credito

In [4]:
base_risco_credito = Orange.data.Table('risco_credito_regras.csv') # o arquivo é diferente, na parte de risco
                                                                   # tem uma c#risco, isso dai é p o Orange saber
                                                                   # que ali é a classe e atrás sao os previsores 👍👍

In [5]:
base_risco_credito

[[ruim, alta, nenhuma, 0_15 | alto],
 [desconhecida, alta, nenhuma, 15_35 | alto],
 [desconhecida, baixa, nenhuma, 15_35 | moderado],
 [desconhecida, baixa, nenhuma, acima_35 | alto],
 [desconhecida, baixa, nenhuma, acima_35 | baixo],
 ...
]

In [7]:
base_risco_credito.domain

[historia, divida, garantias, renda | risco]

In [8]:
cn2 = Orange.classification.rules.CN2Learner()
regras_risco_credito = cn2(base_risco_credito)     # criar as regras

In [9]:
for regra in regras_risco_credito.rule_list:
  print(regra)

IF renda==0_15 THEN risco=alto 
IF historia==boa AND divida!=alta THEN risco=baixo 
IF historia==boa AND garantias!=nenhuma THEN risco=baixo 
IF historia==boa AND renda!=15_35 THEN risco=baixo 
IF historia==boa THEN risco=moderado 
IF divida==alta THEN risco=alto 
IF historia!=desconhecida THEN risco=moderado 
IF garantias==adequada THEN risco=baixo 
IF renda==15_35 THEN risco=moderado 
IF historia==desconhecida THEN risco=baixo 
IF TRUE THEN risco=alto 


In [10]:
# historia boa, divida alta, garantias nenhuma, renda > 35           // nao precisa fazer a conversao p atributo categorico p numero
# historia ruim, divida alta, garantias adequada, renda < 15         // pq o algoritmo consegue trabalhar com atributo assim

previsoes = regras_risco_credito([['boa', 'alta', 'nenhuma', 'acima_35'], ['ruim', 'alta', 'adequada', '0_15']])
previsoes       # baixo e alto

array([1, 0])

In [13]:
base_risco_credito.domain.class_var.values # 0 = alto, 1 = baixo, 2 = moderado

('alto', 'baixo', 'moderado')

In [15]:
for i in previsoes:
  #print(i)
  print(base_risco_credito.domain.class_var.values[i])

baixo
alto


# base credit - 97.40% ACC

In [16]:
base_credit = Orange.data.Table('credit_data_regras.csv')     # 'i#clientd' significa q vai ignorar o id do cliente

In [17]:
base_credit.domain

[income, age, loan | default]

In [18]:
base_dividida = Orange.evaluation.testing.sample(base_credit, n=0.25) # dividir a base de dados de teste

In [19]:
base_dividida # mostra cada uma da base de dados

([[63453.2, 23.9157, 10668.4 | 1],
  [31422.7, 53.8981, 1686.84 | 0],
  [67802.7, 49.038, 13443.5 | 0],
  [34163.6, 45.7827, 6617.4 | 0],
  [39474, 42.3013, 6034.15 | 0],
  ...
 ],
 [[26934.2, 35.979, 4113.3 | 0],
  [39246.5, 48.3221, 919.115 | 0],
  [64780.9, 20.2281, 8402.42 | 1],
  [44379.7, 50.0887, 1814.34 | 0],
  [66008.4, 19.4274, 9189.61 | 1],
  ...
 ])

In [20]:
base_dividida[0]

[[63453.2, 23.9157, 10668.4 | 1],
 [31422.7, 53.8981, 1686.84 | 0],
 [67802.7, 49.038, 13443.5 | 0],
 [34163.6, 45.7827, 6617.4 | 0],
 [39474, 42.3013, 6034.15 | 0],
 ...
]

In [21]:
base_dividida[1]

[[26934.2, 35.979, 4113.3 | 0],
 [39246.5, 48.3221, 919.115 | 0],
 [64780.9, 20.2281, 8402.42 | 1],
 [44379.7, 50.0887, 1814.34 | 0],
 [66008.4, 19.4274, 9189.61 | 1],
 ...
]

In [22]:
base_treinamento = base_dividida[1]
base_teste = base_dividida[0]

In [23]:
len(base_treinamento), len(base_teste)

(1500, 500)

In [24]:
cn2 = Orange.classification.rules.CN2Learner()
regras_credit = cn2(base_treinamento)

In [25]:
for regra in regras_credit.rule_list:
  print(regra)

IF age>=34.9257164876908 THEN default=0 
IF loan<=2495.13299137587 AND income>=20568.891305010802 THEN default=0 
IF loan>=7708.31562497011 AND age>=18.1760434475727 THEN default=1 
IF income>=55931.6545771435 AND income>=58132.4712652713 THEN default=0 
IF loan>=5659.509277874139 AND loan>=6360.15489740922 THEN default=1 
IF income>=48790.1324336417 AND income>=50895.8103414625 THEN default=0 
IF loan>=5836.56338145928 THEN default=1 
IF income>=45360.716274008206 THEN default=0 
IF income<=31702.3342987522 AND loan>=3665.88089899456 THEN default=1 
IF loan>=4849.33378471958 AND age>=18.9473467003554 THEN default=1 
IF income>=24857.6948815025 AND income>=38042.0841556042 THEN default=0 
IF age>=34.1016539284028 THEN default=0 
IF age>=32.516422502230895 THEN default=1 
IF income>=22089.8374845274 AND age>=19.716176087502397 THEN default=0 
IF income>=38011.7266485468 THEN default=1 
IF income>=32197.6207010448 THEN default=0 
IF income<=20113.2534933276 THEN default=1 
IF income<=206

In [26]:
previsoes = Orange.evaluation.testing.TestOnTestData(base_treinamento, base_teste, [lambda testdata: regras_credit])

In [27]:
previsoes

In [30]:
Orange.evaluation.CA(previsoes)

array([0.978])